In [2]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import json
import snowflake.connector
import pandas as pd

In [3]:
load_dotenv()
os.environ['OPENAI_API_KEY']= os.getenv('OPENAI_API_KEY')
client = OpenAI()

In [4]:
def snowflake_connector():
    # Create Snowflake connection
    conn = snowflake.connector.connect(
        user=os.getenv('SNOWFLAKE_USER'),
        password=os.getenv('SNOWFLAKE_PASSWORD'),
        account=os.getenv('SNOWFLAKE_ACCOUNT'),
        warehouse=os.getenv('SNOWFLAKE_WAREHOUSE'),
        database=os.getenv('SNOWFLAKE_DATABASE')
    )
    
    return conn

In [5]:
conn = snowflake_connector()
cursor = conn.cursor()
cursor.execute("""
    select *
    from analytics.trusted.invoiceadvance_invoice_advance_decision;
    """
    )
df = cursor.fetch_pandas_all()

In [24]:
exception_df = df[(df['EXCEPTION_DETAIL_TYPE'].isna()==False) & (df['EXCEPTION_DETAIL_COMMENT']!='')].groupby('EXCEPTION_DETAIL_COMMENT')[['INVOICE_ADVANCE_ID']].count().reset_index()

In [36]:
list_exception = exception_df['EXCEPTION_DETAIL_COMMENT'].to_list()

In [ ]:
prompt = f"""
    Based on following list of exception motives; return an unique list of exception, deduplicating similars
****List
{list_exception}
"""


    Based on following list of exception motives; return an unique list of exception, deduplicating similars
****List
['+ sobregiro', 'Aprobado x Sonny', 'Duplicada', 'Es mail connection', 'Es sin cesion', 'Esta dentro de la los 10 días', 'Excepción aprobada por Sonny es Core Ops', 'Factura cancelada', 'Factura duplicada', 'Factura duplicasa', 'Factura por vencer', 'Facturas aparecen pagadas en portal', 'Indicación del cliente', 'La subirán cuando la confirmen', 'Línea incorrecta', 'No se registran en portal, cliente las subirá cuando se registren', 'No tiene línea', 'Pod incluido en la factura 9616', 'Polimeros', 'Se aprobo plazo por 45 dias. motor de reglas no  toma la actualizacion', 'Se aprobo sobregiro para pasar a factoraje', 'Se cancelan por indicación del cliente', 'Solicitud del cliente', 'Tema de plazo (Collections)', 'Tema de plazo Collections', 'Tenemos Tocken', 'Tenemos tocken', 'Tocken', 'VoBo para sacar facturas por vencer', 'Ya se financió', 'comentario de pago indica 

In [42]:
response = client.responses.create(
    model='gpt-4',
    input=prompt
)

response.output

[ResponseOutputMessage(id='msg_689f4fde246c81988008044760e0f10e0ecf0c414fe15390', content=[ResponseOutputText(annotations=[], text="['+ Sobregiro', 'Aprobado x Sonny', 'Duplicada', 'Es Mail Connection', 'Es Sin Cesión', 'Dentro de los 10 Días', 'Excepción Aprobada por Sonny es Core Ops', 'Factura Cancelada', 'Factura Duplicada', 'Factura por Vencer', 'Facturas Aparecen Pagadas en Portal', 'Indicación del Cliente', 'La Subirán Cuando la Confirmen', 'Línea Incorrecta', 'No se Registran en Portal, Cliente Las Subirá Cuando se Registren', 'No Tiene Línea', 'Pod Incluido en la Factura 9616', 'Polimeros', 'Se Aprobó Plazo por 45 Días. Motor de Reglas no Toma la Actualización', 'Se Aprobó Sobregiro para Pasar a Factoraje', 'Se Cancelan por Indicación del Cliente', 'Solicitud del Cliente', 'Tema de Plazo (Collections)', 'Tenemos Tocken', 'VoBo para Sacar Facturas por Vencer', 'Ya se Financió', 'Comentario de Pago Indica que si Salga Adelantos', 'En Correo Menciona que no Tiene esas Facturas', 

In [68]:
a=response.output_text.split(sep=',')

for item in a:
    item.replace('[','')
    item.replace('"','')

a

["['+ Sobregiro'",
 " 'Aprobado x Sonny'",
 " 'Duplicada'",
 " 'Es Mail Connection'",
 " 'Es Sin Cesión'",
 " 'Dentro de los 10 Días'",
 " 'Excepción Aprobada por Sonny es Core Ops'",
 " 'Factura Cancelada'",
 " 'Factura Duplicada'",
 " 'Factura por Vencer'",
 " 'Facturas Aparecen Pagadas en Portal'",
 " 'Indicación del Cliente'",
 " 'La Subirán Cuando la Confirmen'",
 " 'Línea Incorrecta'",
 " 'No se Registran en Portal",
 " Cliente Las Subirá Cuando se Registren'",
 " 'No Tiene Línea'",
 " 'Pod Incluido en la Factura 9616'",
 " 'Polimeros'",
 " 'Se Aprobó Plazo por 45 Días. Motor de Reglas no Toma la Actualización'",
 " 'Se Aprobó Sobregiro para Pasar a Factoraje'",
 " 'Se Cancelan por Indicación del Cliente'",
 " 'Solicitud del Cliente'",
 " 'Tema de Plazo (Collections)'",
 " 'Tenemos Tocken'",
 " 'VoBo para Sacar Facturas por Vencer'",
 " 'Ya se Financió'",
 " 'Comentario de Pago Indica que si Salga Adelantos'",
 " 'En Correo Menciona que no Tiene esas Facturas'",
 " 'Enviaron POD 

In [9]:
# 1. Define a list of callable tools for the model
tools = [
    {
        "type": "function",
        "name": "get_horoscope",
        "description": "Get today's horoscope for an astrological sign.",
        "parameters": {
            "type": "object",
            "properties": {
                "sign": {
                    "type": "string",
                    "description": "An astrological sign like Taurus or Aquarius",
                },
            },
            "required": ["sign"],
        },
    },
]

# Create a running input list we will add to over time
input_list = [
    {"role": "user", "content": "What is my horoscope? I am an Aquarius."}
]

# 2. Prompt the model with tools defined
response = client.responses.create(
    model="gpt-4",
    tools=tools,
    input=input_list,
)

In [19]:
print(response.output[0].type)

function_call


In [22]:
response.output[0].arguments

'{\n  "sign": "Aquarius"\n}'

In [23]:
# Save function call outputs for subsequent requests
function_call = None
function_call_arguments = None
input_list += response.output

for item in response.output:
    if item.type == "function_call":
        function_call = item
        function_call_arguments = json.loads(item.arguments)


def get_horoscope(sign):
    return f"{sign}: Next Tuesday you will befriend a baby otter."


# 3. Execute the function logic for get_horoscope
result = {"horoscope": get_horoscope(function_call_arguments["sign"])}

# 4. Provide function call results to the model
input_list.append({
    "type": "function_call_output",
    "call_id": function_call.call_id,
    "output": json.dumps(result),
})

print("Final input:")
print(input_list)

response = client.responses.create(
    model="gpt-4",
    instructions="Respond only with a horoscope generated by a tool.",
    tools=tools,
    input=input_list,
)

# 5. The model should be able to give a response!
print("Final output:")
print(response.model_dump_json(indent=2))
print("\n" + response.output_text)

Final input:
[{'role': 'user', 'content': 'What is my horoscope? I am an Aquarius.'}, ResponseFunctionToolCall(arguments='{\n  "sign": "Aquarius"\n}', call_id='call_c2ttwCiqhJL4ctvmHmNbeT5z', name='get_horoscope', type='function_call', id='fc_68990d6440c88199862eba35b7f16f9a08b5e4138cfa6fdb', status='completed'), {'type': 'function_call_output', 'call_id': 'call_c2ttwCiqhJL4ctvmHmNbeT5z', 'output': '{"horoscope": "Aquarius: Next Tuesday you will befriend a baby otter."}'}]
Final output:
{
  "id": "resp_68990e715cdc81998d1e1cc79b104fbb08b5e4138cfa6fdb",
  "created_at": 1754861169.0,
  "error": null,
  "incomplete_details": null,
  "instructions": "Respond only with a horoscope generated by a tool.",
  "metadata": {},
  "model": "gpt-4-0613",
  "object": "response",
  "output": [
    {
      "id": "msg_68990e734b4081999e00f9409abca70d08b5e4138cfa6fdb",
      "content": [
        {
          "annotations": [],
          "text": "Your horoscope says: \"Next Tuesday you will befriend a baby

In [24]:
tools = [{
    "type": "function",
    "name": "get_weather",
    "description": "Get current temperature for a given location.",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "City and country e.g. Bogotá, Colombia"
            }
        },
        "required": [
            "location"
        ],
        "additionalProperties": False
    }
}]

stream = client.responses.create(
    model="gpt-4.1",
    input=[{"role": "user", "content": "What's the weather like in Paris today?"}],
    tools=tools,
    stream=True
)

for event in stream:
    print(event)

ResponseCreatedEvent(response=Response(id='resp_689911e3a7d4819bbe9d613ba71d3da80d9a6948c63508c9', created_at=1754862052.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4.1-2025-04-14', object='response', output=[], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[FunctionTool(name='get_weather', parameters={'type': 'object', 'properties': {'location': {'type': 'string', 'description': 'City and country e.g. Bogotá, Colombia'}}, 'required': ['location'], 'additionalProperties': False}, strict=True, type='function', description='Get current temperature for a given location.')], top_p=1.0, background=False, max_output_tokens=None, max_tool_calls=None, previous_response_id=None, prompt=None, prompt_cache_key=None, reasoning=Reasoning(effort=None, generate_summary=None, summary=None), safety_identifier=None, service_tier='auto', status='in_progress', text=ResponseTextConfig(format=ResponseFormatText(type='text'), verbosity='medium'), 